In [1]:
import numpy as np
import matplotlib.pylab as plt
import sys
import word2vec
import os

In [2]:
from attalos.imgtxt_algorithms.correlation.correlation import *

In [3]:
wordvecs_dir = os.environ["WORDVECS_DIR"]
dataset_dir = os.environ["DATASET_DIR"]
octave_eval_dir = os.environ["OCTAVE_EVAL_DIR"]

In [4]:
w2v_vector_file = os.path.join(wordvecs_dir, "text9Bvin.bin")
w2v_model = word2vec.load(w2v_vector_file)

In [5]:
from oct2py import octave
octave.addpath(octave_eval_dir)

In [6]:
%matplotlib inline

---

# Load data

In [7]:
feature_dir = os.path.join(dataset_dir, "alexnet-feats")
data = np.load(os.path.join(feature_dir, "iaprtc_alexfc7.npz"))
with open(os.path.join(feature_dir, "iaprtc_dictionary.txt")) as dictionary:
    D = dictionary.read().splitlines()
with open(os.path.join(feature_dir, "iaprtc_trainlist.txt")) as trainlist:
    train_ims = [ im.split('/')[-1] for im in trainlist.read().splitlines() ]
    train_ims_full = [ im for im in trainlist.read().splitlines() ]
with open(os.path.join(feature_dir, "iaprtc_testlist.txt")) as testlist:
    test_ims_full = [ im for im in testlist.read().splitlines() ]

xTr = data['xTr'].T # xtrain
yTr = data['yTr'].T # ytrain
xTe = data['xTe'].T # xtest
yTe = data['yTe'].T # ytest

In [8]:
print xTr.shape
print yTr.shape
print xTe.shape
print yTe.shape

(17665, 4096)
(17665, 291)
(1962, 4096)
(1962, 291)


---

# Filter out labels not in W2V vocabulary

In [9]:
to_remove = get_invalid_labels(D, w2v_model.vocab)

In [10]:
D = filter(lambda word: word not in to_remove.keys(), D)

In [11]:
yTr = np.delete(yTr, to_remove.values(), axis=1)

In [12]:
yTe = np.delete(yTe, to_remove.values(), axis=1)

In [13]:
print yTr.shape
print yTe.shape

(17665, 288)
(1962, 288)


In [14]:
orig_yTr = yTr
orig_yTe = yTe

---

# Construct label-to-w2v_vocab correlation matrix

In [15]:
W_L = construct_W(w2v_model, D)

In [16]:
W_V = np.load(os.path.join(wordvecs_dir, "w_v_16.npy")) #construct_W(w2v_model, w2v_model.vocab)
#np.save(os.path.join(wordvecs_dir, "w_v_16.npy"), W_V)

---

# Expand vocabulary

In [17]:
expansion_arr = np.load(os.path.join(wordvecs_dir, "expansion_arr.npy")) #np.dot(W_V.T, W_L)
#np.save(os.path.join(wordvecs_dir, "expansion_arr.npy"), expansion_arr)

In [18]:
expanded = np.dot(yTr, nonlinearity(expansion_arr).T) #np.load(os.path.join(wordvecs_dir, "expanded.npy"))
#np.save(os.path.join(wordvecs_dir, "expanded.npy"), expanded)

MemoryError: 

In [ ]:
# Randomly select an image
i=np.random.randint(0, yTr.shape[1])

# Run example
imname = os.path.join(dataset_dir, "images", "images", "%s.jpg" % train_ims_full[i])
print "Looking at the "+str(i)+"th image: "+imname
im=plt.imread(imname)

# Prediction
#threshold = np.percentile(yHat[i], 99)
#top_idxs = [idx for idx in yHat[i].argsort()[::-1] if yHat[i][idx] > threshold]
#ypwords = [d_words[idx] for idx in top_idxs]
ypwords = [w2v_model.vocab[idx] for idx in expanded[i].argsort()[::-1][:10]]
#ypwords = [D[j] for j in yHat[i].argsort()[::-1] [ 0:(yHat[i]>0.2).sum() ] ]
# Truth
ytwords = [D[idx] for idx in np.nonzero(orig_yTr[i])[0]]
#ytwords = [D[j] for j in np.where(yTe[i] > 0.5)[0] ]
plt.imshow(im)

print 'Predicted: '+ ', '.join(ypwords)
print 'Truth:     '+ ', '.join(ytwords)

plt.figure()